In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os 
import ast
import json
import time
from requests.exceptions import ConnectionError

https://stackoverflow.com/questions/62335537/scraping-text-from-kickstarter-projects-return-nothing

In [16]:
base = "/Users/bandy/Downloads/kickstarter data/"
filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))
df_1 = df.drop_duplicates(subset=['id'])
df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])
print(len(df))
print(len(df_1))

392244
203510


C:\Users\bandy\AppData\Local\Temp\ipykernel_18740\2668440155.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])


In [30]:
df_1.head()

,backers_count,blurb,category,converted_pledged_amount,country,country_displayable_name,created_at,creator,currency,currency_symbol,...,static_usd_rate,urls,usd_exchange_rate,usd_pledged,usd_type,friends,is_backing,is_starred,permissions,main_url
0,31,A community science journalism project to crea...,"{""id"":358,""name"":""Photo"",""analytics_name"":""Pho...",11785.0,CA,Canada,1617889236,"{""id"":578024438,""name"":""Canadian Ocean Literac...",CAD,$,...,0.799456,"{""web"":{""project"":""https://www.kickstarter.com...",0.827300,11388.487833,international,NaN,NaN,NaN,NaN,https://www.kickstarter.com/projects/578024438...
1,414,A book that through photos and words tells the...,"{""id"":358,""name"":""Photo"",""analytics_name"":""Pho...",42522.0,US,the United States,1606841261,"{""id"":1211239611,""name"":""Charlie Bennet"",""slug...",USD,$,...,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1.000000,42522.000000,international,NaN,NaN,NaN,NaN,https://www.kickstarter.com/projects/onpausebo...
2,258,A story about the Newspaper Newsroom,"{""id"":358,""name"":""Photo"",""analytics_name"":""Pho...",22735.0,GB,the United Kingdom,1598965823,"{""id"":942340106,""name"":""Noel Bowler"",""is_regis...",GBP,£,...,1.295785,"{""web"":{""project"":""https://www.kickstarter.com...",1.343951,21920.793830,international,NaN,NaN,NaN,NaN,https://www.kickstarter.com/projects/942340106...
3,55,A book of photographic portraits and interview...,"{""id"":358,""name"":""Photo"",""analytics_name"":""Pho...",4761.0,GB,the United Kingdom,1569947362,"{""id"":1980634804,""name"":""Guy Carpenter"",""slug""...",GBP,£,...,1.228530,"{""web"":{""project"":""https://www.kickstarter.com...",1.290349,4533.275331,international,NaN,NaN,NaN,NaN,https://www.kickstarter.com/projects/dalesfolk...
4,132,Bohypsian is an annual visual arts journal foc...,"{""id"":358,""name"":""Photo"",""analytics_name"":""Pho...",8886.0,US,the United States,1526964272,"{""id"":393220950,""name"":""Michelle Magdalena Mad...",USD,$,...,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1.000000,8886.000000,international,NaN,NaN,NaN,NaN,https://www.kickstarter.com/projects/butterfly...


In [3]:
def get_slug(url):
    return re.search('/projects/(.*)\?', url).group(1)

# get the slug from the urls. the slug is the input to the GraphQL API call 
slugs = list(map(get_slug, df_1['main_url'].tolist()))
# while iterating through all projects, add the story and risk to these lists. At the end len of both list 
# should be the same as the len of the dataframe. if all good then add as columns to the dataframe df. 
story_list2 = []
risk_list2 = []

In [4]:
block_size = len(df_1) // 100
num_blocks = len(df_1) // 2035
num_blocks
2 * block_size
print(0 * block_size, 1 * block_size)
print(len(slugs))

0 2035
203510


In [69]:
mac_df = pd.read_csv("data/mac_final.csv")
mac_df.drop(columns=mac_df.columns[:4], axis=1, inplace=True)
# mac_df.head()

In [131]:
# mac_df.drop(columns={'blurb_x','windows_index','blurb_y','windows_story','windows_risk','mac_index'}, axis=1, inplace=True)
mac_df.tail()

,ivan_index,id,mac_story,mac_risk
5699,139920,1438581587,The goal is to reach $1500. This amount will h...,"Once this campaign reaches its goal, I will se..."
5700,139923,334427102,IMPORTANT: I currently can not accept pledges ...,The risks for a project like this include; any...
5701,139924,1490057200,"Hello watch lover, Thank you for visiting our ...",Because of our previous experience of producin...
5702,139925,1902971331,Nosy is designed to maximise your protection f...,We hope that the manufacturing and distributio...
5703,139949,2040110267,"Hi, everyone! My name is Naomi Romero, and I ...",This is my sixth campaign and each campaign do...


In [169]:
mac_df.sort_values(by=['ivan_index'], ascending=False)['ivan_index'].tolist()

[139949,
 139925,
 139924,
 139923,
 139920,
 139907,
 139905,
 139901,
 139898,
 139872,
 139871,
 139870,
 139868,
 139856,
 139850,
 139838,
 139827,
 139823,
 139809,
 139807,
 139793,
 139788,
 139787,
 139786,
 139780,
 139762,
 139757,
 139749,
 139747,
 139746,
 139742,
 139741,
 139733,
 139728,
 139727,
 139721,
 139717,
 139709,
 139703,
 139682,
 139671,
 139650,
 139644,
 139638,
 139635,
 139630,
 139629,
 139625,
 139615,
 139611,
 139605,
 139591,
 139590,
 139569,
 139561,
 139553,
 139547,
 139537,
 139533,
 139528,
 139522,
 139521,
 139520,
 139510,
 139499,
 139478,
 139477,
 139476,
 139471,
 139470,
 139456,
 139443,
 139432,
 139431,
 139430,
 139395,
 139394,
 139386,
 139377,
 139376,
 139368,
 139342,
 139334,
 139333,
 139332,
 139328,
 139317,
 139307,
 139306,
 139303,
 139299,
 139296,
 139295,
 139293,
 139291,
 139284,
 139279,
 139278,
 139275,
 139252,
 139244,
 139243,
 139223,
 139213,
 139198,
 139192,
 139190,
 139189,
 139185,
 139181,
 139162,
 

In [152]:
mac_df.loc[4000,'ivan_index']


136016

In [37]:
correct_range_df = pd.read_csv('data/correct_range.csv')

In [66]:
check_start = correct_range_df[(correct_range_df['windows_index'] >=122106) & (correct_range_df['windows_index'] <= 129999)]
check_start[(check_start['windows_story'].isnull()) & (check_start['windows_risk'].isnull())]

,Unnamed: 0,ivan_index,id,blurb_x,windows_index,blurb_y,windows_story,windows_risk,mac_index,mac_story,mac_risk
11255,133361,133361,1224405542,NaN,125180,NaN,NaN,NaN,133361,NaN,NaN
22341,144447,144447,233264692,Eterotopia - La Maddalena. Atlante di un'occup...,123326,Eterotopia - La Maddalena. Atlante di un'occup...,NaN,NaN,144447,è il libro che raccoglie un'indagine collettiv...,Eterotopia non è solo un collettivo di archite...
23255,145361,145361,515732745,Very few Cyclorama's exist in New Zealand and ...,122620,Very few Cyclorama's exist in New Zealand and ...,NaN,NaN,145361,Cyclorama is the name given to a large solid s...,Risks:\r\n\r\nPotential damage due to unforese...
24141,146247,146247,1366830633,A modern photographic/multimedia exploration o...,122347,A modern photographic/multimedia exploration o...,NaN,NaN,146247,What is Travels with Quigley? Traveling around...,My biggest challenge is this: raising the fund...
34201,156307,156307,91017538,Join the Empire State Sinfonia and New York F...,122116,Join the Empire State Sinfonia and New York F...,NaN,NaN,156307,NaN,NaN
34256,156362,156362,942196610,It might be the worlds first ever podcast rele...,122175,It might be the worlds first ever podcast rele...,NaN,NaN,156362,NaN,NaN
35222,157328,157328,1381711170,Our project is to build an affordable desktop ...,123172,Our project is to build an affordable desktop ...,NaN,NaN,157328,NaN,NaN


In [196]:
missed_out.shape

(7906, 11)

In [331]:
# to_scrape = pd.read_csv('data/to_scrape2.csv')
# to_scrape_windows = to_scrape[(to_scrape["ivan_index"] <= 129999) | (to_scrape["ivan_index"] >= 140000)]
# to_scrape_windows.head()
# remaining_to_scrape_windows = to_scrape_windows[2001:]
remaining_to_scrape_windows.tail(5)

,Unnamed: 0,ivan_index,id,blurb_x,windows_index,blurb_y,windows_story,windows_risk,mac_index,mac_story,mac_risk
13602,162803,162803,1948268393,Decatur Kids Got Talent will be showcasing Dec...,55818,Decatur Kids Got Talent will be showcasing Dec...,NaN,NaN,162803,NaN,NaN
13603,162804,162804,840331313,We are a group of passionate music professiona...,55819,We are a group of passionate music professiona...,NaN,NaN,162804,NaN,NaN
13604,162805,162805,1625216796,A film series centred around a fictitious merc...,55820,A film series centred around a fictitious merc...,NaN,NaN,162805,NaN,NaN
13605,162806,162806,789664876,This is a pilot episode for a series about a t...,55821,This is a pilot episode for a series about a t...,NaN,NaN,162806,NaN,NaN
13606,162807,162807,1732277792,DJS / DANCING / DRAKE,55823,DJS / DANCING / DRAKE,NaN,NaN,162807,NaN,NaN


In [319]:
max(remaining_to_scrape_windows.index.tolist())

13606

In [329]:
# remaining_to_scrape_windows.loc[7705,['windows_index']][0]|
remaining_to_scrape_windows.loc[(7+7705),['windows_index']]

windows_index    88455
Name: 7712, dtype: object

?

In [171]:
# until_139999_df = correct_range_df.merge(mac_df,on='id', how='left')


for i in range(len(mac_df)):
    idx = mac_df.loc[i,'ivan_index']
    correct_range_df.loc[correct_range_df['ivan_index'] == idx,['mac_risk']] = mac_df.loc[i,['mac_']][0]
    

In [330]:
remaining_to_scrape_windows.loc[13606,['windows_index']][0]

55823

In [336]:
# correct_range_df.loc[correct_range_df['ivan_index'] == 139949,['mac_story']] = 'test'
correct_range_df.loc[correct_range_df['windows_index'] == 55823,:]

,Unnamed: 0,ivan_index,id,blurb_x,windows_index,blurb_y,windows_story,windows_risk,mac_index,mac_story,mac_risk
40701,162807,162807,1732277792,DJS / DANCING / DRAKE,55823,DJS / DANCING / DRAKE,there will be dancing and refreshments and mus...,NaN,162807,NaN,NaN


In [183]:
check = correct_range_df[(correct_range_df['ivan_index'] <130000) | (correct_range_df['ivan_index'] > 139999)]
# check.head()
missed_out = check[(check['mac_story'].isnull()) & (check['windows_story'].isnull())
                          & (check['mac_risk'].isnull()) & (check['windows_risk'].isnull())]
missed_out.shape

(7906, 11)

In [277]:
with open('data2/windows_final_story.json') as data:
    windows_patch2_story = json.load(data)
with open('data2/windows_final_risk.json') as data:
    windows_patch2_risk = json.load(data)
windows_patch2_story = windows_patch2_story[1:]
windows_patch2_risk = windows_patch2_risk[1:]

In [ ]:
print(correct_range_df.loc[correct_range_df['windows_index'] == 55821,:].iloc[0,3])
print("=========================================")
print(windows_patch2_risk[-2])

In [340]:
for i in range(len(windows_patch2_risk)):
    idx = remaining_to_scrape_windows.loc[(i+7705),['windows_index']][0]
#     print(idx)
    correct_range_df.loc[correct_range_df['windows_index'] == idx,['windows_risk']] = windows_patch2_risk[i]
    

In [36]:
with open('data2/122106_129999_risk.json') as file:
    new_data_risk = json.load(file)
with open('data2/122106_129999_story.json') as file:
    new_data_story = json.load(file)

In [118]:
# to_scrape[to_scrape.loc['windows_index'] ]['windows_story']
to_scrape.loc[to_scrape['windows_index']== 124256,['windows_risk']]

,windows_risk
132433,Being that it has been six years since we wrot...


In [130]:
for i in filtered:
    to_add = new_data_story[i-122106]
    correct_range.loc[correct_range['windows_index']== i, ['windows_story']] = to_add
# to_scrape[to_scrape['windows_index'] == 124256]

In [133]:
to_scrape2 = correct_range[(correct_range['mac_story'].isnull()) & (correct_range['windows_story'].isnull())
                          & (correct_range['mac_risk'].isnull()) & (correct_range['windows_risk'].isnull())]
print(to_scrape2.shape)
to_scrape2.head()

(13607, 10)


,ivan_index,id,blurb_x,windows_index,blurb_y,windows_story,windows_risk,mac_index,mac_story,mac_risk
124989,124989,1860583945,An app that helps people to learn the names of...,96768,An app that helps people to learn the names of...,NaN,NaN,124989,NaN,NaN
128000,128000,188784532,Thoughtful packaging paired with a beautiful a...,92531,Thoughtful packaging paired with a beautiful a...,NaN,NaN,128000,NaN,NaN
128001,128001,1145357852,A dental floss dispensing toothpaste cap.,92532,A dental floss dispensing toothpaste cap.,NaN,NaN,128001,NaN,NaN
128002,128002,706701929,"Platform for distributing, creating and enabli...",92533,"Platform for distributing, creating and enabli...",NaN,NaN,128002,NaN,NaN
128003,128003,1061673988,My dream is to provide ~100 middle schoolers i...,92535,My dream is to provide ~100 middle schoolers i...,NaN,NaN,128003,NaN,NaN


In [341]:
missed_out = correct_range_df[(correct_range_df['mac_story'].isnull()) & (correct_range_df['windows_story'].isnull())
                          & (correct_range_df['mac_risk'].isnull()) & (correct_range_df['windows_risk'].isnull())]
missed_out.shape

(11, 11)

In [345]:
correct_range_df.head()

,Unnamed: 0,ivan_index,id,blurb_x,windows_index,blurb_y,windows_story,windows_risk,mac_index,mac_story,mac_risk
0,122106,122106,1899023576,Who says technology can’t bring you closer to ...,112900,Who says technology can’t bring you closer to ...,NaN,NaN,122106,Fifteen Exercises in Perception: Walking Poems...,The main obstacle I prepare for is the possibi...
1,122107,122107,1809386509,Help make my dream of dismantling and repurpos...,112901,Help make my dream of dismantling and repurpos...,NaN,NaN,122107,Help make my dream of dismantling and repurpos...,Risks include injury and lack of storage space...
2,122108,122108,1864257751,0.2s Unlock Without Keys | IP65 Waterproof | U...,112902,0.2s Unlock Without Keys | IP65 Waterproof | U...,NaN,NaN,122108,Why choose Chicso？ Keys are easy to lose. Pass...,"The CHICSO design is finished, The challenges ..."
3,122109,122109,776395884,A collection of career-spanning songs recorded...,112903,A collection of career-spanning songs recorded...,NaN,NaN,122109,The Mascot Theory is proud to introduce Live a...,While there can always be challenges with manu...
4,122110,122110,1569189153,A short film crowdfund campaign,112904,A short film crowdfund campaign,NaN,NaN,122110,Reaching our $7000 stretch goal will ensure a ...,"This campaign is an all or nothing endeavor, i..."


In [355]:
cleaned_df = correct_range_df.loc[:,['ivan_index','id','blurb_x']]
cleaned_df['story']=""
cleaned_df['risk']=""

In [384]:
import math
# math.isnan(correct_range_df.loc[0,'windows_story'])
isinstance(correct_range_df.loc[20,'windows_story'],float)

True

In [387]:
import math
for i in range(len(correct_range_df)):
    windows_story = correct_range_df.loc[i,'windows_risk']
    mac_story = correct_range_df.loc[i,'mac_risk']
    if isinstance(windows_story, float) & isinstance(mac_story,str) :
        cleaned_df.loc[i,'risk'] = mac_story
    elif isinstance(windows_story, str) & isinstance(mac_story,float):
        cleaned_df.loc[i,'risk'] = windows_story
    elif isinstance(windows_story, str) & isinstance(mac_story, str):
        cleaned_df.loc[i,'risk'] = windows_story
    else:
        cleaned_df.loc[i,'risk'] = ""


In [397]:
cleaned_df[(cleaned_df.loc[:,'story'] == "") & (cleaned_df.loc[:,'risk'] == "")].shape

(27, 5)

In [394]:
# cleaned_df.rename(columns = {'blurb_x':'blurb'}, inplace = True)


In [395]:
cleaned_df.to_csv('data/bandy.csv')

In [140]:
# instantiate a new Session 
s = requests.Session()
# headers = {
#             'content-type': 'application/json',
#             'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
#            'cookie': '_ksr_session=fc2U7qXXaRN91foNiE53NyU3s181NZO0Ll57xPkYxZ5iyUNgus35a0HwsPBTfViBY%2ByAKbtpRirAVLxOGKzG%2BYMOmsLRBPujZep%2Fca%2B1%2FXzW3xX56VXkh5w6ItYhIctEFifQQhw3rTmvoljyHw%3D%3D--4pK6xBEgChjqgmte--LH4Q1qSnhU%2FYX9JgTzGuSQ%3D%3D;'
#           }
# GET request to kickstarter to get some token needed for later 
# r = s.get("https://www.kickstarter.com", headers=headers)
# BeautifulSoup parses the html returned as a response from the GET request 
# soup = BeautifulSoup(r.text, 'html.parser')
# xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

# PARAMS
counter = 0
block_number = 0

headers = {
            'content-type': 'application/json',
            'x-csrf-token': "HU2HfDz0ztDItrk3wpnUplOK0LVPB_m5mpYUokYLhofoLpxdyhpPWclCA3R-V8mm5fz_h_GwlxRCOEX5-nj6_A",
            'user-agent': "Mozilla/5.0 (Symbian/3; Series60/5.2 NokiaN8-00/012.002; Profile/MIDP-2.1 Configuration/CLDC-1.1 ) AppleWebKit/533.4 (KHTML, like Gecko) NokiaBrowser/7.3.0 Mobile Safari/533.4 3gpp-gba",
            'cookie': '_ksr_session=XeMoOdlfAuvBg3vZkCPtR2KPkjnWtoE%2FGrwY68dNgE13OYvjkIxO0Eiaf4Oay2xzILiIHuLsmpCAH%2B9aBGT01%2B7gNDH6exw0dtPhWF5xVSYtZnJawqJNc8IUuF52GC%2FS7QHp1woosgy6PABnOt0X0ITzAvUEg8wwMmv5e1IBYNCmj8ZiCiUwlU2'
          }

agents = [
         "Mozilla/5.0 (Symbian/3; Series60/5.2 NokiaN8-00/012.002; Profile/MIDP-2.1 Configuration/CLDC-1.1 ) AppleWebKit/533.4 (KHTML, like Gecko) NokiaBrowser/7.3.0 Mobile Safari/533.4 3gpp-gba",
         "Mozilla/5.0 (Linux; Android 5.0; Lenovo S60-a Build/LRX21T) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 YaBrowser/17.3.1.383.00 Mobile Safari/E7FBAF",
         "Mozilla/5.0 (iPhone; CPU iPhone OS 15_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 Instagram 244.0.0.12.112 (iPhone12,1; iOS 15_5; en_US; en-US; scale=2.00; 828x1792; 383361019)",
         "Mozilla/5.0 (Linux; U; Android 2.2.1; en-US; GT-S5830 Build/FROYO) AppleWebKit/534.31 (KHTML, like Gecko) UCBrowser/9.0.1.275 U3/0.8.0 Mobile Safari/534.31",
         "Mozilla/5.0 (Linux; Android 11; Redmi Note 8 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36"
         ]
global_counter = 0
story_list2 = []
risk_list2 = []
# pause for 5 seconds every 5 iterations
for i in range(159200, 159201):
    global_counter += 1
    slug = slugs[i]
    if counter == 5:
        print("TAKING A NAP ZZZ...")
        time.sleep(30)
        counter = 0
    counter += 1
    print(f"--------{slug}------")
    # POST request to GraphQL API situated in kickstarter domain 
    # response from POST request should contain the story and risks
    connect_try = 0
    not_found = True
    while connect_try < 30 and not_found:       
        try:
            r = s.post("https://www.kickstarter.com/graph",
                headers = {
                  'content-type': 'application/json',
                  'x-csrf-token': 'HU2HfDz0ztDItrk3wpnUplOK0LVPB_m5mpYUokYLhofoLpxdyhpPWclCA3R-V8mm5fz_h_GwlxRCOEX5-nj6_A',
                  'User-Agent':"(Linux; Android 11; Redmi Note 8 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Mobile Safari/537.36",
                  'cookie': '_ksr_session=azdqlNP9tzg8jv57ARONRJG4YEYtpR2M7Eq%2FsXkO7Cn1HEnBSb%2B19Qo28f0AtsGIu5pYc7QJ1k2B9x%2Fo3T5Qp2%2BaVRxt7NKNGZjWzoTEXvoNA133L%2BiZxMloFlpEJTfLDDA3BjTPF7rlgiucD9vFQU8K3ccBS1myolw9W9btI2QYvPDnZzHzIqwqcuLyo5x8064YFeNC8mr0comRXRGkknN0ibSxUm%2BA%2FZgfkFLk5DGqzcX4fJc83qdg1ebYA2LAaVZchoTMLJQ3AgmiJpqgf%2FwR9QA%3D--UxLpDUmTGQV0MzzG--3sj6vixlJjRlRhttV4I7Pw%3D%3D'
                },
                json = {
                    "operationName":"Campaign",
                    "variables":{
                        "slug": slug
                    },
                    "query": query
                })
            not_found = False
        except ConnectionError:
            print("Connection Error")
            time.sleep(40)
            connect_try += 1
    try:
        result = r.json()
        print("-------STORY--------")
        story_html = result["data"]["project"]["story"]
        soup = BeautifulSoup(story_html, 'html.parser')
        curr_story = []
        for i in soup.find_all('p'):
            print(i.get_text())
            curr_story.append(i.get_text())
        story_list2.append(" ".join(curr_story) if curr_story else "")

        print("-------RISKS--------")
        print(result["data"]["project"]["risks"])
        risk_list2.append(result["data"]["project"]["risks"] if result["data"]["project"]["risks"] else "")
        print("Counter is", global_counter)
    except:
        print(r)
        break

--------2121528530/atom-real-steel-robots-fights-the-future-is-now------
-------STORY--------
<Response [200]>


In [21]:
len(story_list2)

5

In [21]:
risk_list2 = risk_list2[:150]

## Latest Iteration Ran Till: 29029, Local copy latest: 0_29029

In [15]:
#SAVE LIST IN LOCAL FOR USE LATER. NAME FILE AS {from}_{to]_{story/risk}.json
story_file_name = "data/136000_139999_story" #CHANGE THIS
risk_file_name = "data/136000_139999_risk" #CHANGE THIS 

story_file_name += ".json"
risk_file_name += ".json"

with open(story_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(story_list2, f, indent=2) 

with open(risk_file_name, 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(risk_list2, f, indent=2) 
    
# with open(story_file_name, 'r') as f:
#     story_subset = json.load(f)

In [ ]:
print("risk:", len(risk_list2),"\n", "story:", len(story_list2))

In [ ]:
print("LATEST CALL DF ROW", "\n")
print(df_1.iloc[130001]['blurb'])
with open("/Users/bandy/Downloads/data/130000_132829_story.json") as fin4:
    data = json.load(fin4)
    
print("LATEST STORY IN LIST", "\n")
print(data[0])

print("LATEST RISK IN LIST", "\n")
print(data[1])

**Combining story and risk for windows**

In [51]:
df_1['windows_story'] = ""
df_1['windows_risk'] = ""
df_1.columns.tolist()[41]

'windows_story'

In [163]:
with open('data/161838_162807_risk.json') as file:
    data = json.load(file)

In [164]:
print(df_1.iloc[161838]['blurb'])
print("================================================")
print(data[0])

Made in Germany, automatic movement Sellita 510BHa, sapphire glass, ceramic bezel, ital. manufacture leather strap, 3 y. guarantee
We can guarantee through our many years of experience that the watch will be made in the best quality. All procedures and processes that are needed for the production of premium watches are performed by us since more than two decades.

Currently, we cannot guarantee the exact delivery time 100 percent. This is because, as in other industries, delivery problems are possible as a result of the pandemic.

At the moment we assume a time span of 7 months from the end of the campaign until we can ship the watches to you. The above factors mean that it may take longer.

A successful campaign can result in demand for a colour variant exceeding the limitation. As a result, the supporters (above the 100 limit) choose from the remaining variants. If this happens, we will of course keep you informed as soon as possible.

We will keep you informed.

Backers are responsi

In [165]:
for i in range(len(data)):
    df_1.iloc[161838+i, 42] = data[i]

In [168]:
df_1.to_csv('data/windows_df.csv')